Previously we used functions that are build into the default python language.
Enhanced functionality is available in python via other [modules](https://www.w3schools.com/python/python_modules.asp). For now we will only use packages that are already installed with the default python. We import this additional functionality with following code.

In [1]:
import json
import datetime

json (Java Script Object Notation) is a way to store data in hierarchical order. To deal with data in that format python provides the [json package]( https://docs.python.org/3/library/json.html).

As we will use dates and times we also import the package [datetime](https://docs.python.org/3/library/datetime.html) which provides the needed functionality.

In [2]:
flightsjson = """
{
    "flights":
    [
        {
            "Origin": "FRA",
            "Destination": "NYC",
            "Depdate": "2020-12-24T12:24:00Z",
            "Arrdate": "2020-12-24T18:42:00Z"
        },
        {
            "Origin": "FRA",
            "Destination": "EWR",
            "Depdate": "2020-12-24T13:42:00Z",
            "Arrdate": "2020-12-24T19:22:00Z"
        },
        {
            "Origin": "FRA",
            "Destination": "EWR",
            "Depdate": "2020-12-24T17:42:00Z",
            "Arrdate": "2020-12-24T21:22:00Z"
        }
    ]
}
"""

The anoying colleague again came to us. This time with the json file that is structured as above. He asked us to sort the flights by flight duration time from lowest to highest.

Many ways lead to rome:

## First way: Build in packages
One way to rome only uses the build in packages json and datetime to do the work.
It has the advantage that it will run anywhere you can run python because it has no other dependencies:

In [3]:
flights = json.loads(flightsjson)["flights"]

In [4]:
flights

[{'Origin': 'FRA',
  'Destination': 'NYC',
  'Depdate': '2020-12-24T12:24:00Z',
  'Arrdate': '2020-12-24T18:42:00Z'},
 {'Origin': 'FRA',
  'Destination': 'EWR',
  'Depdate': '2020-12-24T13:42:00Z',
  'Arrdate': '2020-12-24T19:22:00Z'},
 {'Origin': 'FRA',
  'Destination': 'EWR',
  'Depdate': '2020-12-24T17:42:00Z',
  'Arrdate': '2020-12-24T21:22:00Z'}]

In [5]:
for flight in flights:
    dateformat = "%Y-%m-%dT%H:%M:%SZ"
    flight["Depdate"] = datetime.datetime.strptime(flight["Depdate"], dateformat)
    flight["Arrdate"] = datetime.datetime.strptime(flight["Arrdate"], dateformat)
    flight["Flighttime"] = flight["Arrdate"] - flight["Depdate"]

sorted_flights = sorted(flights, key=lambda f: f["Flighttime"])

In [6]:
sorted_flights

[{'Origin': 'FRA',
  'Destination': 'EWR',
  'Depdate': datetime.datetime(2020, 12, 24, 17, 42),
  'Arrdate': datetime.datetime(2020, 12, 24, 21, 22),
  'Flighttime': datetime.timedelta(seconds=13200)},
 {'Origin': 'FRA',
  'Destination': 'EWR',
  'Depdate': datetime.datetime(2020, 12, 24, 13, 42),
  'Arrdate': datetime.datetime(2020, 12, 24, 19, 22),
  'Flighttime': datetime.timedelta(seconds=20400)},
 {'Origin': 'FRA',
  'Destination': 'NYC',
  'Depdate': datetime.datetime(2020, 12, 24, 12, 24),
  'Arrdate': datetime.datetime(2020, 12, 24, 18, 42),
  'Flighttime': datetime.timedelta(seconds=22680)}]

## Second way: pandas
Another way to rome only uses [pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html) to read the json and sort it, that is much faster,
if the json is more then our two example flights
as it will run the computations in a vectorized way. 
Rough estimation: for 1.000 to 10.000.000 this will be the fastest solution

In [7]:
import pandas as pd

In [8]:
flights = json.loads(flightsjson)["flights"]

In [9]:
df = pd.read_json(json.dumps(flights), orient="records", convert_dates=["Depdate", "Arrdate"])

In [10]:
df

,Origin,Destination,Depdate,Arrdate
0,FRA,NYC,2020-12-24 12:24:00+00:00,2020-12-24 18:42:00+00:00
1,FRA,EWR,2020-12-24 13:42:00+00:00,2020-12-24 19:22:00+00:00
2,FRA,EWR,2020-12-24 17:42:00+00:00,2020-12-24 21:22:00+00:00


In [11]:
df.dtypes

Origin                      object
Destination                 object
Depdate        datetime64[ns, UTC]
Arrdate        datetime64[ns, UTC]
dtype: object

In [12]:
df["Flighttime"] = df["Arrdate"] - df["Depdate"]

In [13]:
df = df.sort_values(by="Flighttime")

In [14]:
df

,Origin,Destination,Depdate,Arrdate,Flighttime
2,FRA,EWR,2020-12-24 17:42:00+00:00,2020-12-24 21:22:00+00:00,03:40:00
1,FRA,EWR,2020-12-24 13:42:00+00:00,2020-12-24 19:22:00+00:00,05:40:00
0,FRA,NYC,2020-12-24 12:24:00+00:00,2020-12-24 18:42:00+00:00,06:18:00


## Third Way: Pyspark
Another way is to only use [pyspark](https://spark.apache.org/docs/latest/sql-getting-started.html#tab_python_0) to read the json and sort it.
This has some overhead as it will transfer the data to a cluster and between multiple nodes.
But it will scale even over the biggest table of flights you can find
Rough estimation: usefull for 100.000 and up.
As it deals with distributed computing this is an additional task that I do not recommend for starters, but feel free. 

Google Colab: Pyspark is not available on google colab out of the box, thus uncomment the commented lines in the two cells below (remove the #) and execute it to install pyspark
Databricks: ignore the next two cells

In [ ]:
# !pip install pyspark

In [15]:
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.getOrCreate()

In [16]:
spark.version

'3.0.1'

In [18]:
sparkdf = spark.createDataFrame(flights)

c:\users\u538946\appdata\local\continuum\miniconda3\envs\python_advent_calendar\lib\site-packages\pyspark\sql\session.py:381: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


In [19]:
sparkdf

DataFrame[Arrdate: string, Depdate: string, Destination: string, Origin: string]

In [20]:
sparkdf = sparkdf.withColumn("Arrdate", sparkdf["Arrdate"].cast("timestamp"))
sparkdf = sparkdf.withColumn("Depdate", sparkdf["Depdate"].cast("timestamp"))

In [21]:
sparkdf

DataFrame[Arrdate: timestamp, Depdate: timestamp, Destination: string, Origin: string]

In [ ]:
# first I thought about this, but the returned type interval can unfortunately not be used for sorting here
# sparkdf = sparkdf.withColumn("FlighttimeReadable", sparkdf["Arrdate"] - sparkdf["Depdate"])

In [22]:
sparkdf = sparkdf.withColumn("Flighttime", sparkdf["Arrdate"].cast("long") - sparkdf["Depdate"].cast("long"))

In [23]:
sparkdf.show()

+-------------------+-------------------+-----------+------+----------+
|            Arrdate|            Depdate|Destination|Origin|Flighttime|
+-------------------+-------------------+-----------+------+----------+
|2020-12-24 19:42:00|2020-12-24 13:24:00|        NYC|   FRA|     22680|
|2020-12-24 20:22:00|2020-12-24 14:42:00|        EWR|   FRA|     20400|
|2020-12-24 22:22:00|2020-12-24 18:42:00|        EWR|   FRA|     13200|
+-------------------+-------------------+-----------+------+----------+



In [24]:
sparkdf.dtypes

[('Arrdate', 'timestamp'),
 ('Depdate', 'timestamp'),
 ('Destination', 'string'),
 ('Origin', 'string'),
 ('Flighttime', 'bigint')]

In [25]:
df = sparkdf.sort("Flighttime").toPandas()

In [26]:
df

,Arrdate,Depdate,Destination,Origin,Flighttime
0,2020-12-24 22:22:00,2020-12-24 18:42:00,EWR,FRA,13200
1,2020-12-24 20:22:00,2020-12-24 14:42:00,EWR,FRA,20400
2,2020-12-24 19:42:00,2020-12-24 13:24:00,NYC,FRA,22680
